In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [5]:
#df_P = pd.read_csv(filepath_or_buffer = 'D3_pre.csv', parse_dates = [[1,2]], index_col = False)  
#df_P = df_P.set_index('YYYY/MM/DD_HH:MM:SS')
#df_P.drop(df_P.columns[[0,2,3,4,5,6,9,10,11,12,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40]], axis=1, inplace=True)
df_P = pd.io.parsers.read_csv(filepath_or_buffer = 'D3_filt_10mino3.csv',index_col = 0)
print df_P[:5]

IOError: File D3_filt_10mino3.csv does not exist

In [ ]:
df_P.drop(df_P.columns[[7,8]], axis=1, inplace=True)
print df_P[:5]

####Define the slope function.

In [ ]:
def find_mean_slope_over_time(df_P, int_min, data_col):
    df_P['diff'] = df_P[data_col].diff()
    
    slope = []
    for i in range(int_min,len(df_P['diff'])):
        top = i-int_min
        slope.append(df_P['diff'][top:i].mean())
    
    for i in range(0,int_min):
        slope.insert(i, 'NaN')

    return slope

####Define functions that create features that represent the area under the temp and humidity curves.

In [ ]:
def make_time_integral_features(df_P,int_min, data_col):
    int_min = 90
    area_curve = []

    for i in range(int_min,len(df_P[data_col])):
        top = i-int_min
        area_curve.append(np.trapz(df_P[data_col][top:i]))
    
    for i in range(0,int_min):
        area_curve.insert(i, 'NaN')
    
    return area_curve

###Call the slope functions and add them to the dataframe.

In [ ]:
df_P['o3_slope_90'] = find_mean_slope_over_time(df_P, 90,'e2v03')
df_P['o3_slope_75'] = find_mean_slope_over_time(df_P, 75,'e2v03')
df_P['o3_slope_60'] = find_mean_slope_over_time(df_P, 60,'e2v03')
df_P['o3_slope_45'] = find_mean_slope_over_time(df_P, 45,'e2v03')
df_P['o3_slope_30'] = find_mean_slope_over_time(df_P, 30,'e2v03')
df_P['o3_slope_15'] = find_mean_slope_over_time(df_P, 15,'e2v03')
df_P['o3_slope_5'] = find_mean_slope_over_time(df_P, 5,'e2v03')

df_P['temp_slope_90'] = find_mean_slope_over_time(df_P, 90,'Temp')
df_P['temp_slope_75'] = find_mean_slope_over_time(df_P, 75,'Temp')
df_P['temp_slope_60'] = find_mean_slope_over_time(df_P, 60,'Temp')
df_P['temp_slope_45'] = find_mean_slope_over_time(df_P, 45,'Temp')
df_P['temp_slope_30'] = find_mean_slope_over_time(df_P, 30,'Temp')
df_P['temp_slope_15'] = find_mean_slope_over_time(df_P, 15,'Temp')
df_P['temp_slope_5'] = find_mean_slope_over_time(df_P, 5,'Temp')

df_P['rh_slope_90'] = find_mean_slope_over_time(df_P, 90,'Rh')
df_P['rh_slope_75'] = find_mean_slope_over_time(df_P, 75,'Rh')
df_P['rh_slope_60'] = find_mean_slope_over_time(df_P, 60,'Rh')
df_P['rh_slope_45'] = find_mean_slope_over_time(df_P, 45,'Rh')
df_P['rh_slope_30'] = find_mean_slope_over_time(df_P, 30,'Rh')
df_P['rh_slope_15'] = find_mean_slope_over_time(df_P, 15,'Rh')
df_P['rh_slope_5'] = find_mean_slope_over_time(df_P, 5,'Rh')

###Call the integration functions and add them to the dataframe.

In [ ]:
#df_P['rh_int_105'] = make_time_integral_features(df_P,105,'Rh')
df_P['rh_int_90'] = make_time_integral_features(df_P,90,'Rh')
df_P['rh_int_75'] = make_time_integral_features(df_P,75,'Rh')
df_P['rh_int_60'] = make_time_integral_features(df_P,60,'Rh')
df_P['rh_int_45'] = make_time_integral_features(df_P,45,'Rh')
df_P['rh_int_30'] = make_time_integral_features(df_P,30,'Rh')

#df_P['temp_int_105'] = make_time_integral_features(df_P,105,'Temp')
df_P['temp_int_90'] = make_time_integral_features(df_P,90,'Temp')
df_P['temp_int_75'] = make_time_integral_features(df_P,75,'Temp')
df_P['temp_int_60'] = make_time_integral_features(df_P,60,'Temp')
df_P['temp_int_45'] = make_time_integral_features(df_P,45,'Temp')
df_P['temp_int_30'] = make_time_integral_features(df_P,30,'Temp')

df_P['o3_int_90'] = make_time_integral_features(df_P,90,'e2v03')
df_P['o3_int_75'] = make_time_integral_features(df_P,75,'e2v03')
df_P['o3_int_60'] = make_time_integral_features(df_P,60,'e2v03')
df_P['o3_int_45'] = make_time_integral_features(df_P,45,'e2v03')
df_P['o3_int_30'] = make_time_integral_features(df_P,30,'e2v03')

In [ ]:
#Delete the first 90 rows of the dataframe to remove all NaNs.
df_P = df_P.ix[90:]


In [ ]:
#df_P.drop(df_P.columns[[9]], axis=1, inplace=True)
#print df_P[:5]

In [ ]:
#ozone terms
df_P['O3_sq'] = df_P['e2v03']**2
df_P['O3_cu'] = df_P['e2v03']**3
df_P['ln_O3'] = np.log(df_P['e2v03'])
df_P['ln_o3_int_90'] = np.log(df_P['o3_int_90'].astype(int))
df_P['ln_o3_int_75'] = np.log(df_P['o3_int_75'].astype(int))
df_P['ln_o3_int_60'] = np.log(df_P['o3_int_60'].astype(int))
df_P['ln_o3_int_45'] = np.log(df_P['o3_int_45'].astype(int))
df_P['ln_o3_int_30'] = np.log(df_P['o3_int_30'].astype(int))
df_P['ln_o3_slope_90'] = np.log(df_P['o3_int_90'].astype(int))
df_P['ln_o3_slope_75'] = np.log(df_P['o3_int_75'].astype(int))
df_P['ln_o3_slope_60'] = np.log(df_P['o3_int_60'].astype(int))
df_P['ln_o3_slope_45'] = np.log(df_P['o3_int_45'].astype(int))
df_P['ln_o3_slope_30'] = np.log(df_P['o3_int_30'].astype(int))
df_P['ln_o3_slope_15'] = np.log(df_P['o3_int_15'].astype(int))
df_P['ln_o3_slope_5'] = np.log(df_P['o3_int_5'].astype(int))

#temp terms
df_P['temp_sq'] = df_P['Temp']**2
df_P['temp_cu'] = df_P['Temp']**3
df_P['ln_temp'] = np.log(df_P['Temp'])
df_P['ln_temp_int_90'] = np.log(df_P['temp_int_90'].astype(int))
df_P['ln_temp_int_75'] = np.log(df_P['temp_int_75'].astype(int))
df_P['ln_temp_int_60'] = np.log(df_P['temp_int_60'].astype(int))
df_P['ln_temp_int_45'] = np.log(df_P['temp_int_45'].astype(int))
df_P['ln_temp_int_30'] = np.log(df_P['temp_int_30'].astype(int))
df_P['ln_temp_slope_90'] = np.log(df_P['temp_int_90'].astype(int))
df_P['ln_temp_slope_75'] = np.log(df_P['temp_int_75'].astype(int))
df_P['ln_temp_slope_60'] = np.log(df_P['temp_int_60'].astype(int))
df_P['ln_temp_slope_45'] = np.log(df_P['temp_int_45'].astype(int))
df_P['ln_temp_slope_30'] = np.log(df_P['temp_int_30'].astype(int))
df_P['ln_temp_slope_15'] = np.log(df_P['temp_int_15'].astype(int))
df_P['ln_temp_slope_5'] = np.log(df_P['temp_int_5'].astype(int))

#rh terms
df_P['rh_sq'] = df_P['Rh']**2
df_P['rh_cu'] = df_P['Rh']**3
df_P['ln_rh'] = np.log(df_P['Rh'])
df_P['ln_rh_int_90'] = np.log(df_P['rh_int_90'].astype(int))
df_P['ln_rh_int_75'] = np.log(df_P['rh_int_75'].astype(int))
df_P['ln_rh_int_60'] = np.log(df_P['rh_int_60'].astype(int))
df_P['ln_rh_int_45'] = np.log(df_P['rh_int_45'].astype(int))
df_P['ln_rh_int_30'] = np.log(df_P['rh_int_30'].astype(int))
df_P['ln_rh_slope_90'] = np.log(df_P['rh_int_90'].astype(int))
df_P['ln_rh_slope_75'] = np.log(df_P['rh_int_75'].astype(int))
df_P['ln_rh_slope_60'] = np.log(df_P['rh_int_60'].astype(int))
df_P['ln_rh_slope_45'] = np.log(df_P['rh_int_45'].astype(int))
df_P['ln_rh_slope_30'] = np.log(df_P['rh_int_30'].astype(int))
df_P['ln_rh_slope_15'] = np.log(df_P['rh_int_15'].astype(int))
df_P['ln_rh_slope_5'] = np.log(df_P['rh_int_5'].astype(int))

#ozone/temp interactions
df_P['03_mult_temp'] = df_P['e2v03']*df_P['Temp']
df_P['03_mult_temp_sq'] = (df_P['e2v03']*df_P['Temp'])**2
df_P['03_mult_temp_cu'] = (df_P['e2v03']*df_P['Temp'])**3
df_P['ln_03_mult_temp'] = np.log(df_P['03_mult_temp'])


#ozone/rh interactions
df_P['03_mult_rh'] = df_P['e2v03']*df_P['Rh']
df_P['03_mult_rh_sq'] = (df_P['e2v03']*df_P['Rh'])**2
df_P['03_mult_rh_cu'] = (df_P['e2v03']*df_P['Rh'])**3
df_P['ln_03_mult_rh'] = np.log(df_P['03_mult_rh'])

#temp/rh interactions
df_P['temp_mult_rh'] = df_P['Temp']*df_P['Rh']
df_P['temp_mult_rh_sq'] = (df_P['Temp']*df_P['Rh'])**2
df_P['temp_mult_rh_cu'] = (df_P['Temp']*df_P['Rh'])**3
df_P['ln_temp_mult_rh'] = np.log(df_P['temp_mult_rh'])


#ozone/rh/temp interactions
df_P['03_mult_rh_&_temp'] = df_P['e2v03']*df_P['Rh']*df_P['Temp']
df_P['03_mult_rh_&_temp_sq'] = (df_P['e2v03']*df_P['Rh']*df_P['Temp'])**2
df_P['03_mult_rh_&_temp_cu'] = (df_P['e2v03']*df_P['Rh']*df_P['Temp'])**3
df_P['ln_03_mult_rh_&_temp_cu'] = np.log(df_P['03_mult_rh_&_temp_cu'])

print df_P[:5]

In [ ]:
df_P.to_csv(path_or_buf = 'D3_raw_features.csv')